# AtticTreasures: Build a training library

In [1]:
# this part of AttacTreasures requieres several spotify playlists
# it collects them, and creates a huge df including spotify features
# the final dataframe will be used to build a model and cluster songs

## Import libraries

In [2]:
#import libraries
from bs4 import BeautifulSoup
import pandas as pd
import requests
from random import randrange
import spotipy # install if needed
from spotipy.oauth2 import SpotifyClientCredentials
import os
import re

## Add YOUR Spotity credentials

In [3]:
#Initialize SpotiPy with user credentials
# Remove mine when uploading

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id="9f9d46aed10c480b9b68543a2d15e2c9",
    client_secret="0016f56c1cc641159dc9e19ba85bc682"))

## Functions:

In [4]:
#get a huge collection of song info from spotify:
#first create a collection of song names:
#get playlist, extranct song uris:
 #uri of a playlist with 100 songs from different artists:
ironhack_tunes="spotify:playlist:1kVI7R6OrKNN8Bf8GMJq9q"

In [5]:
#get a huge collection of song info from spotify:
#first create a collection of song names:
#get playlist, extranct song uris:
 #uri of a playlist with 100 songs from different artists:
ironhack_tunes="spotify:playlist:1kVI7R6OrKNN8Bf8GMJq9q"
shower_playlist="spotify:playlist:37i9dQZF1DWSqmBTGDYngZ"
eighties_to_oughts_playlist="spotify:playlist:6xnhcezDUVM4q6CHbX0nTS"
longest_playlist_ever="spotify:playlist:6yPiKpy7evrwvZodByKvM9"
long_playlist="spotify:playlist:0twlW4iDRhKXAJPopD8PP0"
classical="spotify:playlist:7qvZykTVPjvEX2LCcXoHog"
rap="spotify:playlist:37i9dQZF1DX0XUsuxWHRQd"
jazz="spotify:playlist:05Hd48jdQIz3s8WRrvGnzf"
traditional_chinese="spotify:playlist:3DTJZ0wPXLCUKM0R86fXAy"
traditional_african="spotify:playlist:6P911F6Yy5TszDzeTEmsVI"
traditional_jewish="spotify:playlist:3bvTqM2UeFVOh9vKEu3m1Y"
folk="spotify:playlist:37i9dQZF1DWYV7OOaGhoH0"
oldies="spotify:playlist:37i9dQZF1DXa19sXUAHiO1"
old_hits="spotify:playlist:73TfdlGqJ8t9cL5aXutCij"
old_you_miss="spotify:playlist:4LtOm36qQxUb5I1hkXOviA"
malayam="spotify:playlist:6uxTz9a5sF2vZzaUVdko2x"
bollywood="spotify:playlist:4HF4e71bGTMWbJXNInd0vA"
vintatge_tavel="spotify:playlist:37i9dQZF1DXcW9e9oxFsr1"
throwback="spotify:playlist:2oo75mKwtyxkj4UW092Wmh"
old_rap="spotify:playlist:5MbPhwaiwUypPPEjYuI2MY"
old_arabic="spotify:playlist:6ilWtWCxk76Zz674mp61p2"
old_rnb="spotify:playlist:1NV7bgYOlDNl2bwhU56caw"
sound_tracks="spotify:playlist:6sNDAFPubg3k4CuyH1fqrR"

In [6]:
results_1 = sp.user_playlist_tracks("spotify", longest_playlist_ever)

In [7]:
#starting again:
def get_info_from_playlist(playlist_name):

    '''
    get name of song, artists, and uri of the song in a df
    '''
    
    results = sp.user_playlist_tracks("spotify", playlist_name)
    items = results['items']
    song_uris=[]
    songs = []
    artists = []
    song_id=[]
    year_release=[]
    
    #the the first 100 songs
    for song_item in items:
             # print(["track"][0].keys())
        songs.append(song_item["track"]["name"])
        artists.append(song_item["track"]["artists"][0]["name"])
        song_uris.append(song_item["track"]["uri"])
        song_id.append(song_item["track"]["uri"][14:])
        if song_item["track"]["album"]["release_date"]== None:
            year_release.append(1900)
        else:
            year_release.append(song_item["track"]["album"]["release_date"][:4])
    
    #while there is a next, keep collecting songs
    while results['next']:
        results = sp.next(results)
        items.extend(results['items'])
        for song_item in items:
            songs.append(song_item["track"]["name"])
            artists.append(song_item["track"]["artists"][0]["name"])
            song_uris.append(song_item["track"]["uri"])
            song_id.append(song_item["track"]["uri"][14:])
            if song_item["track"]["album"]["release_date"]== None:
                year_release.append(1900)
            else:
                year_release.append(song_item["track"]["album"]["release_date"][:4])
        
    #create a dictionary after the loop:
    songs_dict={
    "song titles":songs,
    "artists":artists,
    "songs uri":song_uris,
    "song id":song_id,
    "release year":year_release
    }

    #Each key becomes a dataframe column name:
    library_df=pd.DataFrame(songs_dict)
    
    return library_df



def get_song_features(song_uri):
    '''
    input a individual song uri
    output a df with a single row,
    and the features in columns
    '''
    features_dict=sp.audio_features(song_uri)
    features_df=pd.DataFrame(features_dict)
    
    return features_df

def get_uris(playlist_name):
    '''
    input a playlist
    output a list of song uris
    the output can be feeded to the function get_song_features
    '''
    #iwithout the next, only works for playlist with less than 100 songs
    playlist = sp.user_playlist_tracks("spotify", playlist_name)
    items = playlist["items"]
    song_uris = []
    
    for song in items:
        song_uris.append(song["track"]["uri"][14:])
    return song_uris


def get_uris_all(playlist_name):
    #with next:
    '''
    input a playlist
    output a list of song uris
    the output can be feeded to the function get_song_features
    '''
    #include the while
    playlist = sp.user_playlist_tracks("spotify", playlist_name)
    items = playlist["items"]
    song_uris = []
    
    for song_item in items:
        song_uris.append(song_item["track"]["uri"][14:])
    
    #while there is a next, keep collecting songs
    while playlist['next']:
        playlist = sp.next(playlist)
        items.extend(playlist['items'])
        for song_item in items:
            song_uris.append(song_item["track"]["uri"][14:])
        
    return song_uris


## Main

In [8]:
# Build the super playlist:

#List of playlist to include:


list_of_playlists=[ironhack_tunes, shower_playlist, eighties_to_oughts_playlist,
                   longest_playlist_ever, classical, rap,jazz, traditional_chinese,
                   traditional_african, traditional_jewish, folk, oldies,old_hits,
                   old_you_miss, malayam, bollywood, vintatge_tavel,throwback,
                  old_rap, old_arabic, old_rnb,sound_tracks]

list_of_df=[]

#append the df with the info (song names, artist and song uri)
#to a list of df:
for l in list_of_playlists:
    list_of_df.append(get_info_from_playlist(l))

#concat the dfs in a big df:
df_total_pl=pd.concat(list_of_df)


#drop duplicates:
final_df=df_total_pl.drop_duplicates()
len(final_df)


# keep the rows that do not(~) contain ":"" if is more effective than the 22 filter
A=final_df[~final_df['song id'].str.contains(r':')]


#from the final_df I extract the column song uri and pass it to get_song features with a loop
song_uris_list=A["song id"].tolist()
list_of_df_names=[]
for uri in song_uris_list:
    #append each df (a single row each), to a list
    list_of_df_names.append(get_song_features(uri))
    
#concat all the dataframes, since the index of each df is 0, ignore index
df_total_features=pd.concat(list_of_df_names, ignore_index=True)

#merge the info df and the feature df using song uri:
df_complete=A.merge(df_total_features, left_on='songs uri', right_on='uri')


In [9]:
len(df_complete)

15283

In [10]:
df_complete.head()

song titles              artists  \
0                                          D.A.N.C.E              Justice   
1  White & Nerdy (Parody of "Ridin'" by Chamillio...  "Weird Al" Yankovic   
2                                            Goliath        Smith & Thell   
3                                          L'Empordà        Sopa De Cabra   
4                            House of the Rising Sun          The Animals   

                              songs uri                 song id release year  \
0  spotify:track:33yAEqzKXexYM3WlOYtTfQ  33yAEqzKXexYM3WlOYtTfQ         2007   
1  spotify:track:60R2v9lheAu3lwZwAFxMZK  60R2v9lheAu3lwZwAFxMZK         2006   
2  spotify:track:0z4kyKuFGmUV7WQ7gB1sAp  0z4kyKuFGmUV7WQ7gB1sAp         2020   
3  spotify:track:1vzugDTXDgZCF0QQrzB5hX  1vzugDTXDgZCF0QQrzB5hX         2010   
4  spotify:track:4mn2kNTqiGLwaUR8JdhJ1l  4mn2kNTqiGLwaUR8JdhJ1l         1987   

   danceability  energy  key  loudness  mode  ...  liveness  valence    tempo  \
0         0.613   0.962    6    -5.070     0  ...    0.8240    0.496  113.015   
1         0.791   0.613    1   -11.628     0  ...    0.0765    0.896  143.017   
2         0.528   0.833    9    -6.190     1  ...    0.0680    0.460  127.998   
3         0.681   0.837    4    -8.907     1  ...    0.0986    0.959  143.472   
4         0.316   0.484    9    -9.110     0  ...    0.0912    0.299  117.363   

             type                      id  \
0  audio_features  33yAEqzKXexYM3WlOYtTfQ   
1  audio_features  60R2v9lheAu3lwZwAFxMZK   
2  audio_features  0z4kyKuFGmUV7WQ7gB1sAp   
3  audio_features  1vzugDTXDgZCF0QQrzB5hX   
4  audio_features  4mn2kNTqiGLwaUR8JdhJ1l   

                                    uri  \
0  spotify:track:33yAEqzKXexYM3WlOYtTfQ   
1  spotify:track:60R2v9lheAu3lwZwAFxMZK   
2  spotify:track:0z4kyKuFGmUV7WQ7gB1sAp   
3  spotify:track:1vzugDTXDgZCF0QQrzB5hX   
4  spotify:track:4mn2kNTqiGLwaUR8JdhJ1l   

                                          track_href  \
0  https://api.spotify.com/v1/tracks/33yAEqzKXexY...   
1  https://api.spotify.com/v1/tracks/60R2v9lheAu3...   
2  https://api.spotify.com/v1/tracks/0z4kyKuFGmUV...   
3  https://api.spotify.com/v1/tracks/1vzugDTXDgZC...   
4  https://api.spotify.com/v1/tracks/4mn2kNTqiGLw...   

                                        analysis_url duration_ms  \
0  https://api.spotify.com/v1/audio-analysis/33yA...      242267   
1  https://api.spotify.com/v1/audio-analysis/60R2...      170640   
2  https://api.spotify.com/v1/audio-analysis/0z4k...      204759   
3  https://api.spotify.com/v1/audio-analysis/1vzu...      159333   
4  https://api.spotify.com/v1/audio-analysis/4mn2...      269907   

  time_signature  
0              4  
1              4  
2              4  
3              4  
4              3  

[5 rows x 23 columns]

## Potential issues, keep an eye on this

In [11]:
#watch for this kind of strings in "song id column".
#if this happens, the whole row is corrupted, it must be dropped
#I dropped 81 out of 13756 (line 22 of 1.4 Main)
#'''
#Arcade+Fire:Funeral:Neighborhood+%231+%28Tunnels%29:288
#The+Black+Keys:Turn+Blue:Bullet+In+the+Brain:256
#The+Blacke+Keys:Turn+Blue:Fever:246
#The+Black+Keys:Turn+Blue:Gotta+Get+Away:182
#The+Black+Keys:Turn+Blue:In+Our+Prime:278
#'''

##watch null years on playlist, especially classical music playlist.
#I have turned all the "None" years into 1900
#in the function get_info_from_playlist

## save files

In [12]:
#the csv are not needed, just stored for back up and debugging

In [13]:
#df_total_features.to_csv('Data/total_features_crazy.csv')

In [14]:
#A.to_csv('Data/total_playlist_info_crazy.csv')

In [15]:
#df_complete.to_csv('Data/greatest_df_crazy.csv')

In [16]:
df_complete.to_pickle("Data/greatest_df_decade.pkl")